In [1]:
import pandas as pd
import numpy as np
import os
import json
import shutil

In [2]:
folder_dataset = 'dataset'

In [3]:
for f in [folder_dataset, folder_dataset+'/annotations', folder_dataset+'/train2017', folder_dataset+'/val2017']:
    if not os.path.exists(f):
        os.makedirs(f)

In [4]:
for folder, name in zip(['val', 'train'], ['europe', 'world']):
    data = {
        'info' : 'Iberian Wolf Dataset',
        "licenses": [
            {"url": "http://creativecommons.org/licenses/by/2.0/","id": 4,"name": "Attribution License"}
        ],
        'images': [],
        'annotations': [],
        'categories': [
            {'supercategory': 'N/A', 'id':0, 'name': 'N/A'},
            {'supercategory': 'animal', 'id':1, 'name': 'wolf'},
            {'supercategory': 'animal', 'id':2, 'name': 'dog'}
        ]   
    }

    annotation = 0
    for type_name, id_category in zip(['','_dog'], [1,2]):
        dataset = pd.read_csv('dataset'+type_name+'_'+name+'.csv')
        for index, row in dataset.iterrows():
            info_row = {
                'id' : (int(row.id)),
                'url' : row.url,
                "width": row.width, 
                "height": row.height, 
                "file_name": str(id_category)+'_'+str(int(row.id))+".png"
            }
            data['images'].append(info_row)
            detections = np.array(row.detections.split(' ')).astype(np.int32)
            for i in range(int(row.wolves)):
                bbox = detections[i*4:(i+1)*4]
                point0 = bbox[:2]
                point1 = bbox[2:]
                point = point0 if point0[0]<point1[0] else point1
                dims = [abs(point0[0]-point1[0]), abs(point0[1]-point1[1])]
                info_annot = {
                    'id' : annotation,
                    "category_id": id_category, 
                    "iscrowd": 0, 
                    "image_id": (int(row.id)),
                    'bbox' : np.concatenate((point, dims), axis=None).tolist(),
                    "area" : float(dims[0] * dims[1])
                }
                annotation =+1
                data['annotations'].append(info_annot)
            shutil.copy(name+type_name+'/'+str(int(row.id))+".png", folder_dataset+'/'+folder+'2017'+'/'+str(id_category)+'_'+str(int(row.id))+".png")
    with open(folder_dataset+'/annotations/instances_'+folder+'2017.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)